In [1]:
import pandas as pd
import numpy as np

In [2]:
doxyDonkeyPosts = pd.read_csv('doxyDonkeyPosts.csv', index_col = False)
doxyDonkeyPosts = doxyDonkeyPosts['Posts'].values.astype('U')

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')
X = vectorizer.fit_transform(doxyDonkeyPosts)

In [5]:
from sklearn.cluster import KMeans

In [6]:
km = KMeans(n_clusters=3, init='k-means++', max_iter= 100, n_init=1, verbose=True)
km.fit(X)

Initialization complete
Iteration  0, inertia 5331.058
Iteration  1, inertia 2703.192
Iteration  2, inertia 2693.290
Iteration  3, inertia 2688.921
Iteration  4, inertia 2687.149
Iteration  5, inertia 2686.032
Iteration  6, inertia 2685.043
Iteration  7, inertia 2684.447
Iteration  8, inertia 2683.959
Iteration  9, inertia 2683.688
Iteration 10, inertia 2683.375
Iteration 11, inertia 2683.045
Iteration 12, inertia 2682.635
Iteration 13, inertia 2681.893
Iteration 14, inertia 2681.173
Iteration 15, inertia 2680.732
Iteration 16, inertia 2680.565
Iteration 17, inertia 2680.427
Iteration 18, inertia 2680.309
Iteration 19, inertia 2680.207
Iteration 20, inertia 2680.132
Iteration 21, inertia 2680.103
Iteration 22, inertia 2680.081
Iteration 23, inertia 2680.070
Iteration 24, inertia 2680.068
Iteration 25, inertia 2680.066
Converged at iteration 25: center shift 0.000000e+00 within tolerance 7.339674e-09


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=3, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=True)

In [7]:
np.unique(km.labels_, return_counts=True)

(array([0, 1, 2], dtype=int32), array([1638,  686,  480]))

In [12]:
text={}
for i, cluster in enumerate(km.labels_):
    oneDocument = doxyDonkeyPosts[i]
    if cluster not in text.keys():
        text[cluster] = oneDocument
    else:
        text[cluster] += oneDocument

In [13]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import nltk

In [23]:
_stopwords = set(stopwords.words('english') + list(punctuation) + ['million', 'billion', 'years', 'millions', 'billions', 'y/y', "''", "``" "'s", "\n"])

In [24]:
keywords = {}
counts = {}
for cluster in range(3):
    word_sent = word_tokenize(text[cluster].lower())
    word_sent = [word for word in word_sent if word not in _stopwords]
    freq = FreqDist(word_sent)
    keywords[cluster] = nlargest(100, freq, key=freq.get)
    counts[cluster] = freq

In [32]:
unique_keys = {}
for cluster in range(3):
    other_clusters = list(set(range(3)) - set([cluster]))
    keys_other_clusters = set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
    unique = set(keywords[cluster]) - keys_other_clusters
    unique_keys[cluster] = nlargest(10, unique, key = counts[cluster].get)

In [33]:
unique_keys

{0: ['ads',
  'video',
  'use',
  'products',
  'product',
  'search',
  'apps',
  'ad',
  'pay',
  'even'],
 1: ['uber',
  'india',
  'chinese',
  'round',
  'capital',
  'funding',
  'raised',
  'investment',
  'valuation',
  'e-commerce'],
 2: ['quarter',
  'share',
  'profit',
  'rose',
  'earnings',
  'analysts',
  'cents',
  'per',
  'trading',
  'fell']}

In [34]:
article = "Target has finalized an investment in Casper Sleep, pumping $75 million into the fast-growing mattress startup in a funding round that will total $100 million or more, according to a source familiar with the deal. Existing Casper investors like Lerer Hippeau Ventures, IVP and NEA are also participating in the round. New investors, in addition to Target, could send the round over $100 million. The investment comes after Target and Casper could not come to terms on an outright acquisition after Target offered to buy the startup for $1 billion. Casper, which is known for its foam mattresses that it ships to customers folded up in a box, last raised $55 million at a valuation of around $500 million in the summer of 2015. The startup received a higher valuation with this new investment, though the exact terms could not be learned. For Target, the investment signals a move to put its money where its mouth is in its attempt to reclaim some of the cool factor that made it a hit among discount retailers for so long through relationships with popular designers and brands."

In [43]:
from sklearn.neighbors import KNeighborsClassifier 
classifier = KNeighborsClassifier(n_neighbors=10)
classifier.fit(X, km.labels_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [44]:
test = vectorizer.transform([article.encode('ascii', errors='ignore')])

In [45]:
test

<1x13221 sparse matrix of type '<class 'numpy.float64'>'
	with 64 stored elements in Compressed Sparse Row format>

In [46]:
classifier.predict(test)

array([1], dtype=int32)